In [1]:
#V2 will check if the path is empty in the csv. 
#If empty and available fill it,not empty not rewrite it

In [2]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os
import csv
import pandas as pd

In [3]:
def GetTheContourRTUTILS(ctfolder,rtfolder,tot_labels,tumor_labels,ln_labels):
    ct_Match = []
    tot_Match, tumor_Match, ln_Match = ([],[],[])
    if len(ctfolder)==0:
        print("No CT")
    for k in range(len(ctfolder)):
        for j in range(len(rtfolder)):
            rtfiles = os.listdir(rtfolder[j])
            for i in range(len(rtfiles)):
                try:
                    rtstruct = RTStructBuilder.create_from(dicom_series_path=ctfolder[k],rt_struct_path=str(rtfolder[j]+rtfiles[i]))
                    for label in tot_labels:
                        if label in rtstruct.get_roi_names():
                            tot_Match.append(str(rtfolder[j]+rtfiles[i]))
                    for label in tumor_labels:
                        if label in rtstruct.get_roi_names():
                            tumor_Match.append(str(rtfolder[j]+rtfiles[i]))
                    for label in ln_labels:
                        if label in rtstruct.get_roi_names():
                            ln_Match.append(str(rtfolder[j]+rtfiles[i]))
                    
                    if len(tot_Match)+len(tumor_Match)+len(ln_Match) == 0:
                        print("No labels Matched: ",rtstruct.get_roi_names())
                    else:
                        ct_Match.append(ctfolder[k])
                        if len(tot_Match) == 0: tot_Match.append(0)
                        if len(tumor_Match) == 0: tumor_Match.append(0)
                        if len(ln_Match) == 0: ln_Match.append(0)
                            
                except Exception as e: pass

        #mask_3d = rtstruct.get_roi_mask_by_name(label)
    return ct_Match,tot_Match,tumor_Match,ln_Match

In [4]:
#GET PATHS AND LABELS
#root_path  = "//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/DICOM_data/DATA_VOLLEDIG_unstructured/"
root_path  = "//zkh/appdata/RTDicom/DAMEproject/LungSABR_data/Dicom data renamed/"
PxList = np.sort(os.listdir(root_path))
#csv_file_path = "//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/Users/Luis/CSVs/CTandRTMatch.csv"
csv_file_path = "//zkh/appdata/RTDicom/DAMEproject/LungSABR_data/CSV - Copy/LUIS_CSV/CTandRTMatch.csv"
#ITV LABELS
tot_labels = ['ITV',"IGTV","IgTV","ITV1","ITV2","ITV3","ITV totaal def","2ITV","ITV_TOT","ITV_6000","ITV_5100","ITV_Totaal","ITV_LBK","ITV_LOK"]
tumor_labels = ['ITVtumor',"ITV_tumor","ITVtumor def","2ITV_tumor","ITV-P","ITVtumorA1","ITV_tumor_LBK"]
ln_labels = ['ITVklieren',"ITV_klier","ITV_Klier","ITVklieren def","2ITV_klier","ITV_n","ITV_klier_LBK"]
fieldnames = ['ID', 'CT', 'Tot','Tumor',"LymphNodes"]
#GTV LABELS
gtv_tot_labels = ['GTV',"GTV1","GTV2","GTV3","GTV totaal def","2GTV","GTV_TOT","GTV_6000","GTV_5100","GTV_Totaal","GTV_LBK","GTV_LOK"]
gtv_tumor_labels = ['GTVtumor',"GTV_tumor","GTVtumor def","2GTV_tumor","GTV-P","GTVtumorA1","GTV_tumor_LBK"]
gtv_ln_labels = ['GTVklieren',"GTV_klier","GTV_Klier","GTVklieren def","2GTV_klier","GTV_n","GTV_klier_LBK"]
fieldnames = ['ID', 'CT', 'Tot','Tumor',"LymphNodes"]




In [5]:
#LOOK for patients already been analized
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    checked_list = df['ID'].tolist()
    print(len(checked_list),"Px already checked,out of",len(PxList))
    

32 Px already checked,out of 350


In [6]:
with open(csv_file_path, 'a', newline='') as csvfile:
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if csvfile.tell() == 0:
        writer.writeheader()

    for Px in PxList:
        AvgCt_fpaths = []
        RTStruct_fpaths = []
        if not os.path.exists(csv_file_path):
            checked_list = []
        if int(Px) in checked_list:
            print("Px already checked")
        else:
            for root, dirs, files in os.walk(root_path+Px, topdown=True):
                for d in dirs:
                    if "%" in d or "Thorax" in d or "Ave" in d or "MIP" in d or ("CT" in d and not "RTSTRUCT" in d):
                        AvgCt_fpaths.append(root+'/'+d+'/')
                    if ('RTSTRUCT' in d or "pproved" in d) or ("PinnPlan" in d and not("RTDOSE" in d)):
                        RTStruct_fpaths.append(root+'/'+d+'/')


            print("Px",Px,"Num CTs",len(AvgCt_fpaths),"Num RTs",len(RTStruct_fpaths))

            ctMatch,rtMatch_tot,rtMatch_tumor,rtMatch_ln = GetTheContourRTUTILS(AvgCt_fpaths,RTStruct_fpaths,gtv_tot_labels,gtv_tumor_labels,gtv_ln_labels)

            if ctMatch ==None:
                print("No Match")
            else: 
                print("Yes Match")
                writer.writerow({'ID':Px,'CT':ctMatch,'Tot':rtMatch_tot,'Tumor':rtMatch_tumor,'LymphNodes':rtMatch_ln})





Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px 0634082 Num CTs 11 Num RTs 3
Yes Match
Px 0681787 Num CTs 11 Num RTs 2


KeyboardInterrupt: 